# Structure-Based Virtual Screening Pipeline (AutoDock Vina)

## Overview
This notebook demonstrates an automated workflow for structure-based virtual screening using AutoDock Vina. It includes ligand preparation, batch docking with affinity ranking, and extraction and conversion of top candidates.

## Workflow Steps
1. Define working directories
2. Prepare ligands and receptor files
3. Batch Docking and Affinity Ranking
4. Extraction and Conversion of Top Candidates

This workflow is designed for reproducible and automated drug discovery studies.


# 1. Define working directories

In [ ]:
import os

print("Current working directory:", os.getcwd())

# 2. Prepare ligands and receptor files

In [ ]:
import subprocess
import os

# Define generic input file and output directory
input_file = 'ligand_library.sdf'
output_dir = 'prepared_ligands_pdbqt'
os.makedirs(output_dir, exist_ok=True)

# Read the input SDF file
with open(input_file, 'r') as f:
    data = f.read()

# Split the SDF file into individual molecules
molecules = data.split('$$$$\n')

# Iterate over each molecule
for molecule in molecules:
    title = molecule.split('\n', 1)[0].strip()

    # Write temporary SDF file
    with open('temp_ligand.sdf', 'w') as f:
        f.write(molecule)

    # Geometry optimisation using MMFF94
    subprocess.run([
        'obminimize',
        '-ff', 'MMFF94',
        '-c', '1e-6',
        '-n', '1000',
        'temp_ligand.sdf'
    ])

    # Convert optimised ligand to PDBQT format
    pdbqt_file = os.path.join(output_dir, f'{title}.pdbqt')

    subprocess.run([
        'obabel',
        '-isdf', 'temp_ligand.sdf',
        '-opdbqt',
        '-h',
        '-O', pdbqt_file,
        '--partialcharge', 'gasteiger'
    ])

# Clean temporary file
os.remove('temp_ligand.sdf')

# 3. Batch Docking and Affinity Ranking

In [ ]:
import subprocess
import os

# Define executable
vina_exe = 'vina'

# Define generic input and output directories
input_dir = 'prepared_ligands_pdbqt'
output_dir = 'docking_results'
os.makedirs(output_dir, exist_ok=True)

# Number of top-scoring ligands to retain
top_n = 10

log_files = []

# Run docking for each ligand
for ligand in os.listdir(input_dir):
    if ligand.endswith('.pdbqt'):

        ligand_path = os.path.join(input_dir, ligand)
        ligand_name = os.path.splitext(ligand)[0]

        output_file = os.path.join(output_dir, f"{ligand_name}_docked.pdbqt")
        log_file = os.path.join(output_dir, f"{ligand_name}_docking.log")

        cmd = [
            vina_exe,
            '--config', 'config.txt',
            '--ligand', ligand_path,
            '--out', output_file,
            '--log', log_file
        ]

        subprocess.run(cmd)
        log_files.append(log_file)

# Extract affinities from log files
affinities = []

for log_file in log_files:
    with open(log_file, 'r') as f:
        for line in f:
            if line.startswith('-----+------------+----------+----------'):
                next(f)
                affinity_line = next(f)
                affinity = float(affinity_line.split()[1])
                affinities.append((log_file, affinity))

# Rank ligands by binding affinity (more negative = better)
affinities.sort(key=lambda x: x[1])

top_molecules = []

for i, (log_file, affinity) in enumerate(affinities[:top_n]):
    ligand_name = os.path.basename(log_file).replace('_docking.log', '')
    print(f"{i+1}. {ligand_name}: {affinity:.2f} kcal/mol")
    top_molecules.append(f"{ligand_name}: {affinity:.2f}")

# Save ranked results
ranking_file = os.path.join(output_dir, 'top_ranked_ligands.txt')

with open(ranking_file, 'w') as f:
    f.write('\n'.join(top_molecules))

print("\nTop-ranked ligands saved to:", ranking_file)

# 4. Extraction and Conversion of Top Candidates

In [ ]:
import os
import subprocess

# Define directories consistent with docking module
docking_dir = 'docking_results'
output_dir = 'top_ranked_structures'
os.makedirs(output_dir, exist_ok=True)

# Read ranked ligands
ranking_file = os.path.join(docking_dir, 'top_ranked_ligands.txt')

top_molecules = []

with open(ranking_file, 'r') as f:
    for line in f:
        top_molecules.append(line.strip())

# Convert selected ligands to SDF format
for molecule in top_molecules:
    ligand_name = molecule.split(':')[0]

    pdbqt_file = os.path.join(docking_dir, f"{ligand_name}_docked.pdbqt")

    if not os.path.exists(pdbqt_file):
        print(f"Warning: Docked file for {ligand_name} not found")
        continue

    sdf_file = os.path.join(output_dir, f"{ligand_name}.sdf")

    cmd = [
        'obabel',
        '-i', 'pdbqt', pdbqt_file,
        '-o', 'sdf',
        '-O', sdf_file
    ]

    proc = subprocess.run(cmd)

    if proc.returncode == 0:
        print(f"Converted {ligand_name} to SDF format")
    else:
        print(f"Error converting {ligand_name}")